In [1]:
import os
import bqplot as bq
from bqplot import *
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from keras.models import Model
from keras.layers import Dense, Input
from IPython.display import display

Using TensorFlow backend.


In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# Visualize encoder setting
# Parameters
learning_rate = 0.01    # 0.01 this learning rate will be better! Tested
training_epochs = 50
batch_size = 256
display_step = 1
# Network Parameters
n_input = 784  # MNIST data input (img shape: 28*28)
# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])
# hidden layer settings
n_hidden_1 = 128
n_hidden_2 = 64
n_hidden_3 = 10
n_hidden_4 = 2
weights = {
    'encoder_h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1],)),
    'encoder_h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2],)),
    'encoder_h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3],)),
    'encoder_h4': tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_4],)),
    'decoder_h1': tf.Variable(tf.truncated_normal([n_hidden_4, n_hidden_3],)),
    'decoder_h2': tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_2],)),
    'decoder_h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_1],)),
    'decoder_h4': tf.Variable(tf.truncated_normal([n_hidden_1, n_input],)),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'encoder_b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'encoder_b4': tf.Variable(tf.random_normal([n_hidden_4])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_3])),
    'decoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b3': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b4': tf.Variable(tf.random_normal([n_input])),
}
def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['encoder_h3']),
                                   biases['encoder_b3']))
    layer_4 = tf.add(tf.matmul(layer_3, weights['encoder_h4']),
                                    biases['encoder_b4'])
    return layer_4
def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
                                biases['decoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['decoder_h4']),
                                biases['decoder_b4']))
    return layer_4



In [4]:
# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [6]:
show_scatter_vol = 2000
# Launch the graph
with tf.Session() as sess:
    # tf.initialize_all_variables() no long valid from
    # 2017-03-02 if using tensorflow >= 0.12
    if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
        init = tf.initialize_all_variables()
    else:
        init = tf.global_variables_initializer()
    sess.run(init)
    encoder_result = sess.run(encoder_op, feed_dict={X: mnist.test.images[:show_scatter_vol]})
    
    sc_x = LinearScale()
    sc_y = LinearScale()

    sc_c1 = ColorScale()
    scatter = Scatter(x=encoder_result[:show_scatter_vol, 0], y=encoder_result[:show_scatter_vol, 1], 
                      color=mnist.test.labels[:show_scatter_vol],
                      scales={'x': sc_x, 'y': sc_y, 'color': sc_c1}, 
                      stroke='black')

    ax_y = Axis(label='Dim 1', scale=sc_y, 
                orientation='vertical', side='left')

    ax_x = Axis(label='Dim 2', scale=sc_x, num_ticks=10, label_location='end')
    ax_c = ColorAxis(scale=sc_c1, tick_format='0.0f', label='Numbers', orientation='vertical', side='right')

    m_chart = dict(top=50, bottom=70, left=50, right=100)

    fg = Figure(axes=[ax_x, ax_c, ax_y], marks=[scatter], fig_margin=m_chart, animation_duration=1000,
           title='AutoEncoder')
    display(fg)
    
    total_batch = int(mnist.train.num_examples/batch_size)
    for epoch in range(training_epochs):
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)  # max(x) = 1, min(x) = 0
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
    
        if epoch % display_step == 0:
            encoder_result = sess.run(encoder_op, feed_dict={X: mnist.test.images[:show_scatter_vol]})
            scatter.x = encoder_result[:show_scatter_vol, 0]
            scatter.y = encoder_result[:show_scatter_vol, 1]
            scatter.color = mnist.test.labels[:show_scatter_vol]
            fg.title = "Auto Encoder, Epoch: %04d, cost=%f" % (epoch+1, c)

In [11]:
(x_train, _), (x_test, y_test) = mnist.load_data()
# pre-processing
x_train = x_train.astype('float32') / 255. - 0.5       
x_test = x_test.astype('float32') / 255. - 0.5         
x_train = x_train.reshape((x_train.shape[0], -1))
x_test = x_test.reshape((x_test.shape[0], -1))
print(x_train.shape)
print(x_test.shape)

(60000, 784)
(10000, 784)


In [15]:
encoding_dim = 2

input_data = Input(shape=(784,))
# encoder layers
encoded = Dense(128, activation='relu')(input_data)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(10, activation='relu')(encoded)
encoder_output = Dense(encoding_dim)(encoded)

# decoder layers
decoded = Dense(10, activation='relu')(encoder_output)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

# construct the autoencoder model
autoencoder = Model(input=input_data, output=decoded, name="autoencoder")
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_18 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_19 (Dense)             (None, 10)                650       
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 22        
_________________________________________________________________
dense_21 (Dense)             (None, 10)                30        
_________________________________________________________________
dense_22 (Dense)             (None, 64)                704       
__________

/home/yvictor/miniconda2/envs/python3/lib/python3.6/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Model` call to the Keras 2 API: `Model(name="autoencoder", inputs=Tensor("in..., outputs=Tensor("de...)`
  '` call to the Keras 2 API: ' + signature)


In [16]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [19]:
encoder = Model(input=input_data, output=encoder_output)

/home/yvictor/miniconda2/envs/python3/lib/python3.6/site-packages/keras/legacy/interfaces.py:86: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  '` call to the Keras 2 API: ' + signature)


In [20]:
encoder.predict(x_test)

array([[-0.66762799,  0.87247801],
       [-0.19309904,  1.17522848],
       [ 0.19175345,  0.93426901],
       ..., 
       [ 0.12041408,  0.92034042],
       [ 0.0447768 ,  0.96103251],
       [-0.14763096,  1.25440109]], dtype=float32)

In [51]:
show_scatter_vol = 1000
sc_x = LinearScale()
sc_y = LinearScale()

sc_c1 = ColorScale()
scatter = Scatter(x=encoded_imgs[:show_scatter_vol, 0], y=encoded_imgs[:show_scatter_vol, 1], 
                  color=y_test[:show_scatter_vol],
                  scales={'x': sc_x, 'y': sc_y, 'color': sc_c1}, 
                  stroke='black')

ax_y = Axis(label='Dim 1', scale=sc_y, 
            orientation='vertical', side='left')

ax_x = Axis(label='Dim 2', scale=sc_x, num_ticks=10, label_location='end')
ax_c = ColorAxis(scale=sc_c1, tick_format='0.0f', label='Numbers', orientation='vertical', side='right')

m_chart = dict(top=50, bottom=70, left=50, right=100)

Figure(axes=[ax_x, ax_c, ax_y], marks=[scatter], fig_margin=m_chart, animation_duration=100,
       title='AutoEncoder')

In [50]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])